In [3]:
# get the preprocessed data from the preprocess file
from preprocess import *

In [4]:
# Read the content of the file
# with open("corpus/test.txt", "r", encoding="utf-8") as file:
#     data = file.read()

# # Split the content into sentences using "\n" as the delimiter
# sentences = data.split("\n")

In [5]:
# import re
# import string
# from nltk.tokenize import word_tokenize
# from nltk.util import ngrams


# def tokenize_sentences(sentences):
#     unigrams = []
#     bigrams = []
#     trigrams = []
#     fourgrams = []

#     emoji_pattern = re.compile(
#         "["
#         "\U0001F600-\U0001F64F"  # emoticons
#         "\U0001F300-\U0001F5FF"  # symbols & pictographs
#         "\U0001F680-\U0001F6FF"  # transport & map symbols
#         "\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         "\U00002702-\U000027B0"
#         "\U000024C2-\U0001F251"
#         "]+",
#         flags=re.UNICODE,
#     )

#     for sentence in sentences:
#         # Convert to lowercase
#         sentence = sentence.lower()

#         # Remove emojis
#         sentence = emoji_pattern.sub(r"", sentence)

#         # Tokenize the sentence
#         tokens = word_tokenize(sentence)

#         # Remove punctuation
#         tokens = [token for token in tokens if token not in string.punctuation]

#         # Generate unigrams
#         unigrams.extend(tokens)

#         # Generate bigrams
#         bigrams.extend(ngrams(tokens, 2))

#         # Generate trigrams
#         trigrams.extend(ngrams(tokens, 3))

#         # Generate fourgrams
#         fourgrams.extend(ngrams(tokens, 4))

#     return unigrams, bigrams, trigrams, fourgrams


# sentences = [
#     "This is a sample sentence.",
#     "Another example sentence here.",
#     "And one more sentence for testing purposes.",
# ]

# unigrams, bigrams, trigrams, fourgrams = tokenize_sentences(sentences)

# print("Unigrams:", unigrams[:10])
# print("Bigrams:", list(bigrams)[:10])
# print("Trigrams:", list(trigrams)[:10])
# print("Fourgrams:", list(fourgrams)[:10])

In [6]:
# for key, value in freq_four.items():
#     print(count());

In [7]:
# freq_uni.keys()

## Model Building

In [27]:

def unigram_addk_probability(
    freq_uni: dict,
    current_uni: tuple,
    k = 0.1
):
    
    def get_n_total_words():
        # tokenized_sent = tokenize_sentences(sentences)
        return len(tokenized_words['sentences'])
    
    uni_gram_count = freq_uni.get(current_uni, 0)
    n_total_words = get_n_total_words()
    n_unique_words = len(freq_uni)
    
    probability = (uni_gram_count + k) / (n_total_words + n_unique_words * k) 
    return probability

In [34]:
def n_gram_addk_probability(
    word: str,
    given_gram: tuple,
    freq_uni: dict,
    freq_previous: dict, 
    freq_current: dict, 
    k = 0.1
):
    
    # new n-gram
    n_gram = list(given_gram)
    n_gram.append(word)
    n_gram = tuple(n_gram)
    
    current_gram_count = freq_current.get(n_gram, 0)
    previous_gram_count = freq_previous.get(given_gram, 0)
    unique_word_count = len(freq_uni)
    # print(current_gram_count)
    # print(previous_gram_count)
    # print(unique_word_count)
    
    return (current_gram_count + k) / (previous_gram_count + unique_word_count * k)

In [42]:

def addk_probability_interpolation(
    word: str,
    previous_tri_gram: tuple,
    freq_uni: dict,
    freq_bi: dict,
    freq_tri: dict,
    freq_four: dict,
    k=0.1,  # Smoothing parameter
    lambda1=0.1,
    lambda2=0.2,
    lambda3=0.3,
    lambda4=0.4,
):
    """
    Estimate the probability of a word being the next word after given previous words using linear interpolation with add-k smoothing.

    Args:
        word: The word for which to calculate the next word probability.
        previous_words: A tuple containing the two previous words.
        unigram_counts: A dictionary with counts of unigrams.
        bigram_counts: A dictionary with counts of bigrams.
        trigram_counts: A dictionary with counts of trigrams.
        fourgram_counts: A dictionary with counts of fourgrams.
        k: Smoothing parameter (default is 1).
        lambda1: Weight for unigram model.
        lambda2: Weight for bigram model.
        lambda3: Weight for trigram model.
        lambda4: Weight for fourgram model.

    Returns:
        The estimated probability of 'word' being the next word after 'previous_words' using linear interpolation with add-k smoothing.
    """
    # Create the unigram, bigram, trigram, and fourgram tuples
    uni_gram = (word,)
    bi_gram = (previous_tri_gram[1], word)
    tri_gram = (previous_tri_gram[0], previous_tri_gram[1], word)
    four_gram = (previous_tri_gram[0], previous_tri_gram[1], previous_tri_gram[2], word)

    # Calculate probabilities for each n-gram model with add-k smoothing
    unigram_prob = unigram_addk_probability(
        freq_uni=freq_uni, 
        current_uni=uni_gram, 
        k=k,
    )
    
    bigram_prob = n_gram_addk_probability(
        word=bi_gram[1], 
        given_gram=bi_gram[:1], 
        freq_uni=freq_uni, 
        freq_previous=freq_uni, 
        freq_current=freq_bi,
        k=k,
    )
    
    trigram_prob = n_gram_addk_probability(
        word=tri_gram[2],
        given_gram=tri_gram[:2],
        freq_uni=freq_uni, 
        freq_previous=freq_bi, 
        freq_current=freq_tri,
        k=k,
    )
    
    fourgram_prob = n_gram_addk_probability(
        word=four_gram[3], 
        given_gram=four_gram[:3],
        freq_uni=freq_uni, 
        freq_previous=freq_tri, 
        freq_current=freq_four,
        k=k,
    )
    
    print(unigram_prob)
    print(bigram_prob)
    print(trigram_prob)
    print(fourgram_prob)

    # Calculate interpolated probability
    probability = (lambda1*unigram_prob) + (lambda2*bigram_prob) + (lambda3*trigram_prob) + (lambda4*fourgram_prob)

    return probability

In [43]:
addk_probability_interpolation(
    'test', 
    ('this', 'is', 'a'),
    freq_uni,
    freq_bi,
    freq_tri,
    freq_four,
)

0.14285714285714288
0.058823529411764705
0.058823529411764705
0.6470588235294118


0.3025210084033614

In [39]:
n_gram_addk_probability('is', ('this' 'is'), freq_uni, freq_uni, freq_bi)

0.14285714285714285

In [15]:
(1 + 0.1) / (1 + 0.7)

0.6470588235294118

In [13]:
unigram_addk_probability(freq_uni, ('this',))

0.14285714285714288

In [14]:
(1 + 0.1) / (7 + 0.7)

0.14285714285714288

In [16]:
unigram_addk_probability(freq_uni, ('iNtelligence'.lower(),))

0.012987012987012988

In [17]:
from collections import Counter
unigram_counts = Counter(unigrams)
bigram_counts = Counter(bigrams)
trigram_counts = Counter(trigrams)
fourgram_counts = Counter(fourgrams)


test = addk_probability_interpolation(
    "it",
    ("for", "testing", "purposes"),
    unigram_counts,
    bigram_counts,
    trigram_counts,
    fourgram_counts,
)

print(test)

NameError: name 'unigrams' is not defined

In [ ]:
def generate_text(
    seed_words,
    length,
    interpolation_function,
    unigram_counts,
    bigram_counts,
    trigram_counts,
    fourgram_counts,
):
    generated_text = list(seed_words)

    for _ in range(length):
        previous_words = tuple(generated_text[-3:])  # Get the last three words
        next_word = generate_next_word(
            interpolation_function,
            previous_words,
            unigram_counts,
            bigram_counts,
            trigram_counts,
            fourgram_counts,
        )
        generated_text.append(next_word)

    return " ".join(generated_text)


def generate_next_word(
    interpolation_function,
    previous_words,
    unigram_counts,
    bigram_counts,
    trigram_counts,
    fourgram_counts,
):
    # Create a list to store predicted words and their probabilities
    predictions = []

    # Iterate through all possible unigrams
    for word in unigram_counts.keys():
        # Calculate the probability of the word being the next word
        probability = interpolation_function(
            word,
            previous_words,
            unigram_counts,
            bigram_counts,
            trigram_counts,
            fourgram_counts,
        )

        # Add the word and its probability to the list
        predictions.append((word, probability))

    # Sort the predictions by probability in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Return the word with the highest probability
    return str(predictions[0][0])
    # return predictions[0][0]


seed_words = ["for", "testing", "purposes",]  # Seed sequence
generated_text = generate_text(
    seed_words,
    100,
    addk_probability_interpolation,
    unigram_counts,
    bigram_counts,
    trigram_counts,
    fourgram_counts,
)
print(generated_text)

for testing purposes purposes this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is is a a sample sample sentence sentence here here this this is


In [ ]:
# build a model using interpolation
def language_model_interpolation():
    return

## Model Evaluation

In [ ]:
def perplexity_interpolation():
    return

## Text Generation